In [6]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow as tf
import os
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


In [8]:
tf.config.set_visible_devices([], 'GPU')

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Crecimiento de memoria configurado para las GPUs.")
    except RuntimeError as e:
        print(f"Error al configurar crecimiento de memoria: {e}")

Crecimiento de memoria configurado para las GPUs.


# BIBLIOTECA HUGGING FACE DE TRANSFORMERS

HuggingFace, es una empresa de IA que ha construido un ecosistema completo de herramientas de código abierto fáciles de usar para el PLN, la visión, y más. El componente central de su ecosistema es la biblioteca Transformers, que permite descargar con facilidad un modelo preentrenado, incluyendo su tokenizador correspondiente y, después, ajustarlo en nuestro propio conjunto de datos, si es necesario. 

Además, la biblioteca es compatible con TensorFlow, y PyTorch. 

La manera más fácil de utilizar la biblioteca Transformers es usar la función transformers.pipeline(): solo tienes que especificar qué tarea quieres hacer y se descarga un modelo preentrenado predeterminado, listo para usar:

In [9]:
#classifier = pipeline("sentiment-analysis", device=-1) #CPU
classifier = pipeline("sentiment-analysis") #GPU
result = classifier("The actors were very convincing.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


El resultado es una lista de Python que contiene un diccionario con la predicción:

In [10]:
result

[{'label': 'POSITIVE', 'score': 0.9998071789741516}]

En este ejemplo, el modelo ha descubierto correctamente que la oración es positiva, con una confianza del 99,98%. Por supuesto, también puedes pasar un lote de oraciones al modelo:

## SESGO E IMPARCIALIDAD

In [11]:
classifier(["I am from India.", "I am from Iraq."])

[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

Como sugiere la salida este clasificador específico adora a los indios, pero tiene prejuicios contra los iraquíes. Puedes probar este código con tu propio país o ciudad. Por lo general, un sesgo tan indeseable viene, en gran parte, de los propios datos de entrenamiento. En este caso, había un montón de oraciones negativas relacionadas con las guerras en lrak en los datos de entrenamiento. Este sesgo se amplificó durante el proceso de ajuste fino, ya que el modelo estaba obligado a elegir entre solo dos clases: positiva o negativa. Si añades una clase neutra al ajustar, el sesgo del país desaparece en su mayor parte. Pero los datos de entrenamiento no son la única fuente del sesgo: la arquitectura del modelo, el tipo de pérdida o regularización utilizada para el entrenamiento, el optimizador; todo ello puede afectar a lo que acaba aprendiendo el modelo.

Entender el sesgo en la IA y mitigar sus efectos negativos sigue siendo un área de investigación activa, pero una cosa está clara: deberás detenerte a pensar antes de lanzarte a desplegar un modelo en producción. Pregúntate qué mal podría causar el modelo, incluso de forma indirecta. Por ejemplo, si las predicciones del modelo se utilizan para decidir si conceder o no a alguien un préstamo, el proceso debería ser justo. Así pues, asegúrate de evaluar el rendimiento del modelo no solo según la media del conjunto de entrenamiento completo, sino también de varios subconjuntos: por ejemplo, puede que descubras que, aunque el modelo funciona muy bien de media, su rendimiento es pésimo para algunas categorías de personas. Puede que también te convenga realizar pruebas contrafactuales: por ejemplo, puede que quieras comprobar que las predicciones del modelo no cambian cuando cambias simplemente el género de una persona. Si el modelo funciona bien de media, resulta tentador pasarlo a producción y ponerse con otra cosa. Pero, en general, si no arreglas esos problemas tú, nadie lo hará, y el modelo acabará causando más mal que bien. La solución depende del problema: puede que haga falta reequilibrar el conjunto de datos, ajustar con un conjunto de datos diferente, cambiar a otro modelo preentrenado, ajustar la arquitectura o los hiperparámetros del modelo, etc.

## MODELOS EN HUGGING FACE

La función pipeline() utiliza un modelo predeterminado para cada tarea. Pero es posible y recomendable especificar de forma manual un modelo diferente. Por ejemplo, podrías utilizar un modelo DistilBERT ajustado en la tarea Multi-Genre Natural Language lnference (MultiNLI), que clasifica dos oraciones en tres clases: contradicción, neutral o consecuencia lógica.

In [12]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
#classifier_mnli = pipeline("text-classification", model=model_name, device=-1) #CPU
classifier_mnli = pipeline("text-classification", model=model_name) #GPU
classifier_mnli("She loves me. [SEP] She loves me not.")

Device set to use cuda:0


[{'label': 'contradiction', 'score': 0.9790192246437073}]

Puedes encontrar los modelos disponibles en https://huggingface.co/models y la lista de tareas en https://huggingface.co/tasks 

La API pipeline es muy simple y conveniente, pero, a veces, necesitas más control. Para casos así, la biblioteca Transformers ofrece muchas clases, incluyendo todo tipo de tokenizadores, modelos, configuraciones, retrollamadas y mucho más. 

Por ejemplo, vamos a cargar el mismo modelo DistilBERT, junto con su tokenizador correspondiente, utilizando las clases TFAutoModelForSequenceClassification y AutoTokenizer:


In [13]:
# Se carga el tokenizador pre-entrenado utilizando el nombre del modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Se carga el modelo pre-entrenado para clasificación de secuencias utilizando el nombre del modelo
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


A continuación, vamos a tokenizar un par de pares de oraciones. En este código, activamos el relleno y especificamos que queremos tensores TensorFlow en vez de listas de Python:

In [14]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!",
                       "Joe lived for a very long time. [SEP] Joe is old."],
                      padding=True, return_tensors="tf")


La salida es un tensor (de tipo BatchEncoding), que contiene secuencias de identificadores de tokens, además de una máscara que contiene ceros para los tokens de relleno:

In [15]:
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

En vez de pasar "Sentence 1 [SEP] Sentence 2" al tokenizador, podemos pasar, de forma equivalente. una tupla: ("Sentence 1", "Sentence 2").

In [16]:
token_ids = tokenizer([("I like soccer.", "We all love soccer!"),
                       ("Joe lived for a very long time.", "Joe is old.")],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

A continuación, podemos pasar directamente este objeto BatchEncoding al modelo; devuelve un objeto TFSequenceClassifierOutput que contiene sus logits de clase predichas.

Los logits son los valores sin procesar que una red neuronal produce como salida antes de aplicar una función de activación final, como softmax o sigmoide, para convertirlos en probabilidades. En el contexto de clasificación de redes neuronales, los logits representan la puntuación que el modelo asigna a cada clase antes de normalizarlas.

In [17]:
outputs = model(token_ids)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-2.1123824,  1.178678 ,  1.4101027],
       [-0.0147834,  1.0962467, -0.991995 ]], dtype=float32)>, hidden_states=None, attentions=None)

Por último, podemos aplicar la función de activación softmax para convertir estas logits en probabilidades de clase y utilizar la función argmax() para predecir la clase con la probabilidad más alta para cada par de oraciones de entrada:

In [18]:
Y_probas = tf.keras.activations.softmax(outputs.logits)
Y_probas

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.016197  , 0.4352351 , 0.5485679 ],
       [0.22655988, 0.6881723 , 0.08526786]], dtype=float32)>

In [19]:
Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred  # 0 = contradicción 1 = consecuecia lógica, 2 = neutral

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1])>

Si quieres ajustar este modelo en tu propio conjunto de datos, puedes entrenarlo del modo habitual en Keras. Sin embargo, puesto que el modelo genera como salida logits en vez de probabilidades, debes utilizar la función de pérdida tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), en vez de la pérdida "sparse_categorical_crossentropy" habitual.

Además, el modelo no soporta entradas BatchEncoding durante el entrenamiento, así que debes usar su atributo data para obtener un diccionario corriente en su lugar:

In [20]:
sentences = [("Sky is blue", "Sky is red"), ("I love her", "She loves me")]
X_train = tokenizer(sentences, padding=True, return_tensors="tf").data
y_train = tf.constant([0, 2])  # contradiccion, neutral
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2)

Epoch 1/2


I0000 00:00:1738917065.433273    8117 service.cc:148] XLA service 0x739e980143c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738917065.433328    8117 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-02-07 09:31:05.451843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1738917065.596815    8117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 15s 15s/step - loss: 0.8088 - accuracy: 0.5000
Epoch 2/2
1/1 [==============================] - 0s 312ms/step - loss: 1.4327 - accuracy: 0.5000


Hugging Face también ha creado una biblioteca Datasets que puedes utilizar para descargar con facilidad un conjunto de datos estándar (como IMDb) o uno personalizado y utilizarlo para ajustar tu modelo. Es similar a TensorFlow Datasets, pero también proporciona herramientas para realizar tareas de preprocesamiento comunes sobre la marcha, como el enmascaramiento. La lista de conjuntos de datos está disponible en https://huggingface.co/datasets 

Para aprender más, puedes acceder a https://huggingface.co/docs para ver la documentación, que incluye varios cuadernos tutoriales, vídeos, la API completa y más. 